In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from .neural_network import NeuralNetworkModel, ChessDataset, ChessEvaluationNeuralNetwork

In [6]:
def eval_to_output(eval: str):
    if eval.startswith('#+'):
        return 10000.0
    elif eval.startswith('#-'):
        return -10000.0
    else:
        return float(eval)

In [ ]:
df = pd.read_csv("../data/chess_data.csv")

X = df['FEN'].apply(NeuralNetworkModel.fen_to_feature_array).tolist()
X = np.array(X, dtype=np.float32)
y = df['Evaluation'].apply(eval_to_output).astype(np.float32).values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = ChessDataset(X_train, y_train)
test_dataset = ChessDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [10]:
model = ChessEvaluationNeuralNetwork(input_size=X.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 927219.9411
Epoch 2, Loss: 850644.8584
Epoch 3, Loss: 735805.7372
Epoch 4, Loss: 614551.8796
Epoch 5, Loss: 512876.8745
Epoch 6, Loss: 448385.6092
Epoch 7, Loss: 394669.8345
Epoch 8, Loss: 361629.4191
Epoch 9, Loss: 328408.9472
Epoch 10, Loss: 298896.5376
Epoch 11, Loss: 269858.6813
Epoch 12, Loss: 257559.2605
Epoch 13, Loss: 240418.3916
Epoch 14, Loss: 222882.6354
Epoch 15, Loss: 217015.7272
Epoch 16, Loss: 202830.9199
Epoch 17, Loss: 196762.8443
Epoch 18, Loss: 182347.3733
Epoch 19, Loss: 177794.7490
Epoch 20, Loss: 166972.8774


In [11]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
rmse_error = np.sqrt(avg_test_loss)
print(f"\nTest RMSE: {rmse_error:.4f}")


Test RMSE: 675.1118


In [12]:
torch.save(model.state_dict(), 'neural_network.pth')